In [1]:
import os 
import sys 
import json 
import numpy as np 
from skimage.io import imread, imsave
from skimage.draw import draw 
from skimage.transform import resize
import matplotlib.pyplot as plt 
import albumentations as A 
import cv2 
import random 
import glob
import torch 
import shutil
import time
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from models.unet import UNet
import glob
import torch
from albumentations.pytorch import ToTensorV2
import torch.utils.data
from torch.utils.data.dataloader import default_collate

In [3]:
json_dict = dict()
for index in os.listdir(data_format):
    if "ipynb" in index:
        continue
    json_dict[index] = dict()
    path = os.path.join(data_format,index)
    path = os.path.join(path, "images/")
    for file in os.listdir(path):
        if ".json" in file:
            json_path = path + file
        elif "small" in file:
            continue
        else:
            img_path = path + file
    with open(json_path) as f: 
        annotations = json.load(f)
    for ele in annotations["shapes"]:
        json_dict[index][ele["label"]] = ele["points"]
    json_dict[index]["path"] = img_path
    json_dict[index]["shape"] = (annotations["imageHeight"], annotations["imageWidth"])

In [4]:
def img_max_length(img, max_length, mode):
    """
    Resize Image to Max_Length For UNET 
    """
    height, width = img.shape[:2]
    aspect = width/float(height)
    
    if (mode == "img"):
        kwargs = {"order": 1, "anti_aliasing":True, "preserve_range": False}
    elif (mode == "mask"):
        kwargs = {"order": 0, "anti_aliasing": False, "preserve_range": True}
    else:
        return(1)
    if (height > max_length or width > max_length):
        if (aspect < 1):
            res = int(aspect * max_length)
            img_small = resize(img, (max_length, res), **kwargs)
        elif (aspect > 1):
            res = int(max_length/aspect)
            img_small = resize(img, (res, max_length), **kwargs)
        elif (aspect == 1):
            img_small = resize(img, (max_length, max_length), **kwargs)
        else:
            return(1)
        if (mode == "mask"):
            img_small = img_small.astype("uint8")
        return img_small
    else:
        return(img)

In [145]:
for index in tqdm(json_dict.keys()):
    img_dir = os.path.join(os.path.join(data_small,index),"image")
    if not os.path.exists(img_dir):
        os.makedirs(img_dir, exist_ok = True)
    polygon = dict()
    img_longest_size = 512
    image_path = json_dict[index]["path"]
    image = imread(image_path)
    height, width = image.shape[:2]
    image_path = os.path.join(img_dir,"image.jpg")
    img_small = img_max_length(image, img_longest_size, mode = "img")
    imsave(image_path, img_small)
    classes = ["leg", "wound"]
    for cl in classes:
        if not cl in json_dict[index].keys():
            continue
        polygon[cl] = json_dict[index][cl]
    mask_dir = os.path.join(os.path.join(data_small,index),"mask")
    if not os.path.exists(mask_dir):
        os.makedirs(mask_dir, exist_ok=True)
    for cl in classes: 
        if not cl in json_dict[index].keys():
            continue
        mask = np.zeros([height, width], dtype = np.float32)
        h = np.array(polygon[cl])[:,1]
        w = np.array(polygon[cl])[:,0]
        rr,cc = draw.polygon(h,w)
        mask[rr,cc] = 1
        mask_path = os.path.join(mask_dir, cl +"_small" + ".jpg")
        mask_small = img_max_length(mask, img_longest_size, mode = "mask")
        imsave(mask_path, mask_small)

  0%|          | 0/372 [00:00<?, ?it/s]

Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0

Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0

Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0

Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0

Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0

Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0

In [5]:
def my_collate(batch):
    batch = list(filter(lambda x : x is not None, batch))
    return default_collate(batch)
class WoundDataset(torch.utils.data.Dataset):
    def __init__(self, index, images_dir, class_type, transform = None):
        self.images_dir = images_dir
        self.index = index
        self.class_type = class_type
        self.transform = transform
    def __len__(self):
        return len(self.index)
    def __getitem__(self, idx):
        path = os.path.join(self.images_dir, self.index[idx])
        img_path = os.path.join(path, "image")
        image = cv2.imread(glob.glob(img_path + "/*.jpg")[0])
        mask_path = os.path.join(path,"mask")
        if len(os.listdir(mask_path)) == 1:
            mask_leg = cv2.imread(glob.glob(mask_path + "/leg*")[0])[:,:,0]
            mask_leg[mask_leg == 2] = 1
            mask_wound = np.zeros((image.shape[:2]), dtype = np.float64)
        else:
            mask_leg = cv2.imread(glob.glob(mask_path + "/leg*")[0])[:,:,0]
            mask_leg[mask_leg == 2] = 1
            mask_wound = cv2.imread(glob.glob(mask_path + "/wound*")[0])[:,:,0]
            mask_wound[mask_wound == 2] = 1

        if self.transform is not None:
            transformed = self.transform(image=image, mask=np.stack([mask_leg,mask_wound],axis = 2))
            image = transformed["image"]
            mask = transformed["mask"]
        return image, mask

In [6]:
#function to return data loaders
def data_loaders(dataset_train, dataset_valid, bs = 5):

    loader_train = torch.utils.data.DataLoader(
        dataset_train,
        batch_size=bs,
        shuffle=True,
        collate_fn=my_collate,
#         drop_last=True,
    )
    loader_valid = torch.utils.data.DataLoader(
        dataset_valid,
        batch_size=bs,
#         drop_last=False,
        shuffle=False,
        collate_fn=my_collate,
    )

    return {"train": loader_train, "valid": loader_valid}
#define augmentations
train_transform = A.Compose([
    A.Resize(512, 512, always_apply=True),
    A.RandomCrop(height=200, width = 200, p=0.2),
    A.PadIfNeeded(min_height=512, min_width=512, border_mode=cv2.BORDER_CONSTANT, 
                  always_apply=True),
    A.VerticalFlip(p=0.2),              
    A.Blur(p=0.2),
    A.RandomRotate90(p=0.2),
    A.ShiftScaleRotate(p=0.2, border_mode=cv2.BORDER_CONSTANT),
    A.RandomBrightnessContrast(p=0.2),
    A.RandomSunFlare(p=0.2, src_radius=200),
    A.RandomShadow(p=0.2),
    A.RandomFog(p=0.2),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2(transpose_mask=True)
]
)

val_transform = A.Compose([
    A.Resize(512, 512, always_apply=True), 
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)), 
    ToTensorV2(transpose_mask=True)
]
)

In [7]:
indices = []
for index in os.listdir(data_small):
    if "ipynb" in index:
        continue
    indices.append(index)

train_data = WoundDataset(indices[:300],data_small, ["leg", "wound"],train_transform)
test_data = WoundDataset(indices[300:], data_small, ["leg", "wound"],val_transform)


In [8]:

class DiceLoss(torch.nn.Module):
    def __init__(self):
        super(DiceLoss, self).__init__()
        self.smooth = 1.0

    def forward(self, y_pred, y_true):
        assert y_pred.size() == y_true.size()
        y_pred = y_pred[:, :].contiguous().view(-1)
        y_true = y_true[:, :].contiguous().view(-1)
        intersection = (y_pred * y_true).sum()
        
        
        dsc = (2.*intersection+ elf.smooth)/(y_pred.sum()+y_true.sum()+self.smooth)
        return 1. - dsc

In [9]:
#define datasets and load network from file
loaders = data_loaders(train_data, test_data)
device = torch.device("cpu" if not torch.cuda.is_available() else "cuda:0")
print(torch.cuda.is_available())
unet = UNet(3, 2)
unet.to(device)

dsc_loss = DiceLoss()
best_validation_dsc = 1.0

lr = 0.01
epochs = 1

optimizer = torch.optim.Adam(unet.parameters(), lr=lr)

best_val_true = []
best_val_pred = []
epoch_loss = {"train": [], "valid": []}

True


In [10]:
#training loop 
for epoch in range(epochs):
    print('-' * 100)
    since = time.time()
    #best_model_wts = copy.deepcopy(unet.state_dict())
    
    for phase in ["train", "valid"]:
        
        if phase == "train":
            unet.train()
        else:
            unet.eval()

        epoch_samples = 0
        running_loss = 0
        
        for i, data in enumerate(loaders[phase]):
            x, y_true = data
            x, y_true = x.to(device), y_true.to(device)
            
            epoch_samples += x.size(0)
            
            optimizer.zero_grad()
            
            with torch.set_grad_enabled(phase == "train"):
                y_pred = unet(x)
                loss = dsc_loss(y_pred, y_true)
                running_loss += loss.item()
                    
                if phase == "train":
                    loss.backward()
                    optimizer.step()

        epoch_phase_loss = running_loss/epoch_samples
        epoch_loss[phase].append(epoch_phase_loss)
    
#         if phase == "valid" and epoch_phase_loss < best_validation_dsc:
#             best_validation_dsc = epoch_phase_loss
#             print("Saving best model")
#             best_model_wts = copy.deepcopy(unet.state_dict())
#             best_val_true = y_true.detach().cpu().numpy()
#             best_val_pred = y_pred.detach().cpu().numpy()
        
#     time_elapsed = time.time() - since
    print("Epoch {}/{}: Train loss = {:4f} --- Valid loss = {:4f} --- Time: {:.0f}m {:.0f}s".format(epoch + 1, epochs, epoch_loss["train"][epoch], epoch_loss["valid"][epoch], time_elapsed // 60, time_elapsed % 60))

print("Best validation loss: {:4f}".format(best_validation_dsc))
torch.save(best_model_wts, os.path.join(data_dir, "unet.pt"))

----------------------------------------------------------------------------------------------------


RuntimeError: CUDA out of memory. Tried to allocate 80.00 MiB (GPU 0; 3.95 GiB total capacity; 3.05 GiB already allocated; 80.50 MiB free; 3.10 GiB reserved in total by PyTorch)

In [75]:
for i,k in loaders["train"]:
    print(i)

IndexError: list index out of range

In [42]:
for x in loaders["train"]:
    print(x)

IndexError: list index out of range